# Local Simulation Trading Demo

演示使用 deltafq 在本地完成一个基于实时行情的模拟交易流程，步骤：

1. 创建模拟账号
2. 获取行情数据
3. 模拟买入1：买入价格与行情数据近似
4. 模拟买入2：买入价格低于行情数据
5. 持续等待：tick 撮合交易，观察订单执行
6. 一定时间后结束
7. 打印订单、账户、持仓信息等

> 说明：Notebook 假设在项目根目录下运行，如路径不同，请按需调整 `project_root`。

In [1]:
pip install deltafq==0.7.0  # 需要>=0.7.0


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
import time
import threading

from deltafq.live import EventEngine
from deltafq.live.event_engine import EVENT_TICK
from deltafq.live.gateway_registry import create_data_gateway, create_trade_gateway
from deltafq.live.models import OrderRequest

In [4]:
# 1. 创建模拟账户，并设置交易参数

symbol = "000001.SS"          # 要交易的标的
initial_capital = 100_000.0   # 初始资金

# 事件引擎：负责转发 tick
event_engine = EventEngine()

# 行情网关（yfinance）
data_gw = create_data_gateway("yfinance", interval=5.0)

# 模拟交易网关（本地撮合，内部使用 match_on_tick=True 的 ExecutionEngine）
trade_gw = create_trade_gateway(
    "paper",
    initial_capital=initial_capital,
    commission=0.001,
)

# 连接网关
assert trade_gw.connect(), "paper 交易网关连接失败"
assert data_gw.connect(), "yfinance 行情网关连接失败"

print("模拟账号已创建：")
print("  初始资金:", initial_capital)
print("  当前持仓:", trade_gw._engine.position_manager.get_all_positions())

[22:47:23] YFinanceDataGateway  >>> INFO     >>> Initialized YFinanceDataGateway with interval: 5.0s
[22:47:23] OrderManager         >>> INFO     >>> Initializing order manager
[22:47:23] PositionManager      >>> INFO     >>> Initializing position manager
[22:47:23] ExecutionEngine      >>> INFO     >>> Initializing paper trading execution engine with capital: 100000.0
[22:47:23] YFinanceDataGateway  >>> INFO     >>> Connected to yfinance
模拟账号已创建：
  初始资金: 100000.0
  当前持仓: {}


In [5]:
# 2. 获取行情数据，并定义撮合逻辑

last_tick_price = {}
first_live_tick = threading.Event()


def on_tick_match(tick):
    """将 tick 喂给撮合引擎：ExecutionEngine(match_on_tick=True) 会在 on_tick 里根据限价单和 tick 价格撮合。"""
    trade_gw._engine.on_tick(tick)


def on_tick_print(tick):
    """打印实时行情，记录最近价格，跳过 warmup 历史数据。"""
    if getattr(tick, "source", None) == "yf_warmup":
        return
    last_tick_price[tick.symbol] = tick.price
    first_live_tick.set()
    ts = tick.timestamp.strftime("%H:%M:%S") if tick.timestamp else ""
    print(f"[Live] {tick.symbol} -> {tick.price} | Vol: {tick.volume or '-'} @ {ts}")


# 注册事件处理
event_engine.on(EVENT_TICK, on_tick_match)
event_engine.on(EVENT_TICK, on_tick_print)

# 将行情推入 EventEngine
data_gw.set_tick_handler(lambda t: event_engine.emit(EVENT_TICK, t))

# 订阅并启动行情
data_gw.subscribe([symbol])
data_gw.start()

print(f"等待 {symbol} 的首个实时 tick ...")
if not first_live_tick.wait(timeout=60):
    raise RuntimeError("60 秒内无实时 tick，退出")

print("首个实时 tick 已到，最近价：", last_tick_price.get(symbol))

[22:53:05] YFinanceDataGateway  >>> INFO     >>> Subscribed & Warmed up 000001.SS (239 bars)
[22:53:05] YFinanceDataGateway  >>> INFO     >>> Starting yfinance polling
等待 000001.SS 的首个实时 tick ...
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:07首个实时 tick 已到，最近价： 4075.916748046875

[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:12
[22:53:18] PositionManager      >>> INFO     >>> Position updated: 000001.SS -> 10
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:18
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:23
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:29
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:35
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:41
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:46
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:52
[Live] 000001.SS -> 4075.916748046875 | Vol: 3837648752 @ 14:53:57
[Live] 000001.S

In [6]:
# 3. 模拟买入1：买入价格与行情数据近似

last_price = last_tick_price.get(symbol)
if last_price is None:
    raise RuntimeError("尚未记录到最近价")

# 稍微高于当前价一点，提高尽快成交的概率
limit_price_1 = round(last_price * 1.001, 2)

req1 = OrderRequest(
    symbol=symbol,
    quantity=10,                # 买入 10 单位
    price=limit_price_1,
    order_type="limit",
)
order_id_1 = trade_gw.send_order(req1)
print(f"模拟买入1提交：{order_id_1} | {symbol} buy 10 @ {limit_price_1} （接近当前价）")

[22:53:15] OrderManager         >>> INFO     >>> Order created: ORD_000001
[22:53:15] ExecutionEngine      >>> INFO     >>> Order submitted (pending): ORD_000001, symbol: 000001.SS, price: 4079.99, quantity: 10
模拟买入1提交：ORD_000001 | 000001.SS buy 10 @ 4079.99 （接近当前价）


In [7]:
# 4. 模拟买入2：买入价格低于行情数据

limit_price_2 = round(last_price * 0.95, 2)   # 比当前价低约 5%

req2 = OrderRequest(
    symbol=symbol,
    quantity=10,
    price=limit_price_2,
    order_type="limit",
)
order_id_2 = trade_gw.send_order(req2)
print(f"模拟买入2提交：{order_id_2} | {symbol} buy 10 @ {limit_price_2} （低于当前价，可能长时间挂单）")

[22:53:26] OrderManager         >>> INFO     >>> Order created: ORD_000002
[22:53:26] ExecutionEngine      >>> INFO     >>> Order submitted (pending): ORD_000002, symbol: 000001.SS, price: 3872.12, quantity: 10
模拟买入2提交：ORD_000002 | 000001.SS buy 10 @ 3872.12 （低于当前价，可能长时间挂单）


In [8]:
# 5. 持续等待：tick 撮合交易，观察订单执行

print("\n开始观察撮合结果（每秒检查一次，打印新增成交）...")

printed_trade_count = 0
observe_seconds = 60   # 观察 60 秒，你可按需调整

start_ts = time.time()
try:
    while time.time() - start_ts < observe_seconds:
        time.sleep(1)
        trades = trade_gw._engine.trades
        if len(trades) > printed_trade_count:
            for t in trades[printed_trade_count:]:
                print(f"  -> Filled: {t['type']} {t['symbol']} qty={t['quantity']} @ {t['price']}")
            printed_trade_count = len(trades)
except KeyboardInterrupt:
    print("手动中断观察。")


开始观察撮合结果（每秒检查一次，打印新增成交）...
  -> Filled: buy 000001.SS qty=10 @ 4075.916748046875


In [9]:
# 6. 一定时间后结束（停止行情轮询）

data_gw.stop()
print("行情轮询已停止。")

[22:55:09] YFinanceDataGateway  >>> INFO     >>> Stopped yfinance polling
行情轮询已停止。


In [10]:
# 7. 打印订单、账户、持仓信息

engine = trade_gw._engine

print("\n=== 账户汇总 ===")
print("当前现金余额:", round(engine.cash, 2))
print("当前持仓:", engine.position_manager.get_all_positions())
print("成交记录条数:", len(engine.trades))

print("\n=== 订单明细 ===")
for o in engine.order_manager.get_order_history():
    print(
        f"{o['id']} | {o['symbol']} qty={o['quantity']} "
        f"type={o['order_type']} price={o['price']} status={o['status']}"
    )

print("\n=== 成交明细 ===")
for t in engine.trades:
    print(
        f"{t['timestamp']} | {t['type']} {t['symbol']} qty={t['quantity']} "
        f"@ {t['price']} commission={t['commission']}"
    )


=== 账户汇总 ===
当前现金余额: 59200.07
当前持仓: {'000001.SS': 10}
成交记录条数: 1

=== 订单明细 ===
ORD_000001 | 000001.SS qty=10 type=limit price=4079.99 status=executed
ORD_000002 | 000001.SS qty=10 type=limit price=3872.12 status=pending

=== 成交明细 ===
2026-02-05 14:53:18.379264 | buy 000001.SS qty=10 @ 4075.916748046875 commission=40.759167480468754
